In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import torch_geometric.nn as gnn

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
np.random.seed(1)

Using cpu device


## Input Format

Neighbor Sampling: [torch_geometric.loader.NeighborLoader](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/neighbor_loader.html)

HeteroData: [torch_geometric.data.HeteroData](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.HeteroData.html)

In [2]:
from torch_geometric.data import Data    ## The input form is not adj_Mtx, since its too costly for large graph

edge_index = torch.Tensor([         ## edge pairs: from--to
    [0,  0,  1,  1,  2],
    [1,  5,  0,  2,  1]
])

x = torch.Tensor([                   ## 5 node embedding(dim=7)
    [1., 0., 0., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 0.],
])

y = torch.Tensor([1,2,3,4,5])        ## node labels

train_mask = [True, True, True, True, False]   ## use part of the graph when training?

Data(x=x, edge_index=edge_index, y = y, train_mask = train_mask)

Data(x=[5, 7], edge_index=[2, 5], y=[5], train_mask=[5])

In [ ]:
from torch_geometric.loader import NeighborLoader

x = torch.randn(8, 32)          # Node features  [num_nodes, num_features]
y = torch.randint(0, 4, (8, ))  # Node labels   [num_nodes]

edge_index = torch.tensor([                       #   0  1
    [2, 3, 3, 4, 5, 6, 7],                        #  / \/ \
    [0, 0, 1, 1, 2, 3, 4]],                       # 2  3  4
)                                                 # |  |  |
data = Data(x=x, y=y, edge_index=edge_index)      # 5  6  7

loader = NeighborLoader(              ## extract subgraph from the full graph: k-hoop neighours of some nodes
    data,
    input_nodes=torch.tensor([0]),    ## nodes to be sampled
    num_neighbors=[1,1],              ## 1-hoop: 1 neighours, 2-hoop: 1 neighours
    batch_size=1,                     ## how many input_nodes to be taken in one batch
    replace=False,
    shuffle=False)

batch = next(iter(loader))

batch.n_id                    ##    [0, 2, 5]     original id: node + k-hoop nodes selected
batch.edge_index              ## [[1, 2],[0, 1]]  index based on n_id above (subgraph)
batch.n_id[batch.edge_index]  ## [[2, 5],[0, 2]]  index based on original id (full graph)

## GNN Trials

Tutorial: https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html

```
Aggr(Neighbour_info) --->  NN ---> Node Representation ---> other Tasks

```

In [3]:
gnn_layer_by_name = {
    "GCN": gnn.GCNConv,
    "GAT": gnn.GATConv,
    "GraphConv": gnn.GraphConv
}           

### Node Level  (Supervised Learning)

To predict the label of each node in a single graph

```
node_embeddings(Original)  ----[GNN Layers]----> node_embeddings(Class)

```

input:: node_embeddings, node_labels, adj_Mtx

In [4]:
data = torch_geometric.datasets.Planetoid(root=r'L:\Datasets', name="Cora")[0]  ## A single graph with 2780 nodes, mask = [True, True, False, ...]
data, min(data.y),max(data.y)                                                   ##    node class y = 7        if mask: x[mask], data.y[mask]

(Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]),
 tensor(0),
 tensor(6))

In [5]:
class NodeGNN(nn.Module):
    def __init__(self, c_in, c_out, activate = True):    ## node_embd_dim  -->  node_class
        super().__init__()
        self.activate = activate
        self.layers = nn.ModuleList([
            gnn.GCNConv(c_in,32),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),

            gnn.GCNConv(32,c_out),
            # nn.Sigmoid()
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            if isinstance(layer, gnn.MessagePassing):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        if self.activate:
            x = F.softmax(x, dim=1)
        return x


def train_step(model, lossfn, optimizer, data):           ## data is one graph, i.e. batch=1
    model.train()
    optimizer.zero_grad()
    x, edge_index, y = data.x.to(device), data.edge_index.to(device), data.y.to(device)   
    pred = model(x, edge_index)          
    loss = lossfn(pred,y)
    loss.backward()
    optimizer.step()
    acc = (pred.argmax(dim=-1) == y).sum().float() / x.size()[0]
    return loss.item(), acc.item()



node_model = NodeGNN(1433,7).to(device)
node_lossfn = nn.CrossEntropyLoss()
node_optimizer = torch.optim.SGD(node_model.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)

for epoch in range(200):
    loss, acc = train_step(node_model, node_lossfn, node_optimizer, data)
    if epoch % 50 == 0:
        print("Epoch {} acc:{}  loss:{}".format(epoch,acc,loss))

Epoch 0 acc:0.17208272218704224  loss:1.94431734085083
Epoch 50 acc:0.30206793546676636  loss:1.829414963722229
Epoch 100 acc:0.5243722200393677  loss:1.6869120597839355
Epoch 150 acc:0.7119645476341248  loss:1.551342248916626


### Graph Level(Supervised Learning)


```
node_embeddings(Original)  ----[GNN Layers]----> node_embeddings ----[Aggr]----> graph_embedding ----[Classifier]----> graph_label

```
input:: node_embeddings, adj_Mtx, graph_labels

Aggr see: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/pool.html

In [6]:
## How does batch data look like? torch_geometric.loader.DataLoader() will concat several Graph's adjacency matrix into one.
dataset = torch_geometric.datasets.TUDataset(root=r'L:\Datasets', name="MUTAG")   ## A set of small molecules
train_loader = torch_geometric.loader.DataLoader(dataset, batch_size=64, shuffle=True)

for batch_data in train_loader:
    break
batch_data, batch_data.batch  ,set(dataset.y.numpy())

(DataBatch(edge_index=[2, 2544], x=[1154, 7], edge_attr=[2544, 4], y=[64], batch=[1154], ptr=[65]),
 tensor([ 0,  0,  0,  ..., 63, 63, 63]),
 {0, 1})

In [7]:
class GraphGNN(nn.Module):
    def __init__(self, c_in, c_out):   
        super().__init__()
        self.nodeGNN = NodeGNN(c_in, 100, activate = False)
        self.binaryClassifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(100, c_out),
            nn.Sigmoid()
        )

    def forward(self, x, edge_index, batch_index):
        x = self.nodeGNN(x, edge_index)
        x = gnn.global_mean_pool(x, batch_index)   ## Aggr: mean
        x = self.binaryClassifier(x)
        return x.squeeze(-1)


graph_model = GraphGNN(7,1).to(device)
graph_lossfn = nn.BCELoss()
graph_optimizer = torch.optim.AdamW(graph_model.parameters(), lr=1e-2, weight_decay=0.0)


def train_step(model, lossfn, optimizer, batch_data):           ## data is one graph, i.e. batch=1
    model.train()
    optimizer.zero_grad()
    x, edge_index, batch_index, y = batch_data.x.to(device), batch_data.edge_index.to(device), batch_data.batch.to(device), batch_data.y.to(device)
    pred = model(x, edge_index, batch_index)     
    loss = lossfn(pred,y.type(torch.float))
    loss.backward()
    optimizer.step()
    acc = (pred.ge(1/2) == y).sum().float() / (batch_data.batch[-1].item() + 1)  ## /current batch size
    return loss.item(), acc.item()


for epoch in range(200):
    for batch_data in train_loader:
        loss, acc = train_step(graph_model, graph_lossfn, graph_optimizer, batch_data)
    if epoch % 50 == 0:
        print("Epoch {}'s final batch -- acc:{}  loss:{}".format(epoch,acc,loss))


Epoch 0's final batch -- acc:0.6666666865348816  loss:0.6265289783477783
Epoch 50's final batch -- acc:0.7166666388511658  loss:0.49427902698516846
Epoch 100's final batch -- acc:0.8333333134651184  loss:0.42077288031578064
Epoch 150's final batch -- acc:0.7833333611488342  loss:0.4838566482067108


### Edge Level

Predict possible new edges (assume there are missing edges)

Example2: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

Idea: if 2 nodes' representation are similar, probably there will be a link